## Identify WDPAs in each gadm1


This notebook performs a spatial join between the WDPAs dataset and the gadm1 administrative boundaries to identify which WDPAs belong to each of the regions defined in the gadm1.

#### Import packages

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
import requests as re
from copy import deepcopy

#### Connect to arcgis online with credentials saved in .env file

In [2]:
env_path = ".env"
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

aol_password = env['ARCGIS_GRETA_PASS']
aol_username = env['ARCGIS_GRETA_USER']

gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)

#### Read files

In [29]:
## read tables saved in local as geodataframes
# wdpa = gpd.read_file('zip:///Users/sofia/Documents/HE_Data/WDPA_FILTERED_20210615_FILTERED_nomarine_with_oecm_wdpa.zip')
gadm1 = gpd.read_file('zip:///Users/sofia/Documents/HE_Data/gadm36_level1.zip')

In [138]:
# ## read gadm1 calling it from arcgis online
# item_id = '3f89b653de41428a8166d51f383f0950'
# item = gis.content.get(item_id)
# flayer = item.layers[0]
# gadm1 = pd.DataFrame.spatial.from_layer(flayer) # this is a sdf

In [139]:
# ## convert gadm1 from dataframe to geodataframe
# gadm1 = gpd.GeoDataFrame(gadm1, geometry='SHAPE')
# gadm1.crs
# gadm1 = gadm1.set_crs(epsg=3857) # set crs
# gadm1 = gadm1.to_crs(epsg=4326) # project to wdpa crs

#### Select only relevant columns

In [30]:
gadm = gadm1[['GID_0','NAME_0','NAME_1','MOL_ID','geometry']]
gadm.rename(columns={'MOL_ID':'MOL_IDg'},inplace=True)

wd = wdpa[['MOL_ID','ISO3','NAME','ORIG_NA','STATUS','GOV_TYP','DESIG','DESIG_E','DESIG_T','IUCN_CA','AREA_KM','geometry']] 

/opt/anaconda3/envs/vizz38/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [31]:
wd.head()

,MOL_ID,ISO3,NAME,ORIG_NA,STATUS,GOV_TYP,DESIG,DESIG_E,DESIG_T,IUCN_CA,AREA_KM,geometry
0,1,NZL,Boulder Beach,Boulder Beach / WWF Block,Designated,Federal or national ministry or agency,Stewardship Area,Stewardship Area,National,III,1.136031,"MULTIPOLYGON (((170.62746 -45.89520, 170.62824..."
1,2,NZL,Ferndale,Ferndale,Designated,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.748492,"POLYGON ((173.97642 -41.19258, 173.97595 -41.1..."
2,3,NZL,Broughton Bay,Broughton Bay,Designated,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.031907,"POLYGON ((173.95691 -41.20976, 173.95580 -41.2..."
3,4,NZL,Kaipupu Point,Kaipupu Point,Designated,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.270855,"POLYGON ((174.00241 -41.27336, 174.00312 -41.2..."
4,5,NZL,Catlins Conservation Park,Catlins Conservation Park,Designated,Federal or national ministry or agency,Stewardship Area,Stewardship Area,National,III,8.412168,"MULTIPOLYGON (((169.30645 -46.57949, 169.30754..."


#### Spatial join

In [33]:
gwd = gpd.sjoin(gadm,wd, how='right')

In [34]:
gwd.head()

,index_left,GID_0,NAME_0,NAME_1,MOL_IDg,MOL_ID,ISO3,NAME,ORIG_NA,STATUS,GOV_TYP,DESIG,DESIG_E,DESIG_T,IUCN_CA,AREA_KM,geometry
0,2214.0,NZL,New Zealand,Otago,2215.0,1,NZL,Boulder Beach,Boulder Beach / WWF Block,Designated,Federal or national ministry or agency,Stewardship Area,Stewardship Area,National,III,1.136031,"MULTIPOLYGON (((170.62746 -45.89520, 170.62824..."
1,2210.0,NZL,New Zealand,Marlborough,2211.0,2,NZL,Ferndale,Ferndale,Designated,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.748492,"POLYGON ((173.97642 -41.19258, 173.97595 -41.1..."
2,2210.0,NZL,New Zealand,Marlborough,2211.0,3,NZL,Broughton Bay,Broughton Bay,Designated,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.031907,"POLYGON ((173.95691 -41.20976, 173.95580 -41.2..."
3,2210.0,NZL,New Zealand,Marlborough,2211.0,4,NZL,Kaipupu Point,Kaipupu Point,Designated,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.270855,"POLYGON ((174.00241 -41.27336, 174.00312 -41.2..."
4,2214.0,NZL,New Zealand,Otago,2215.0,5,NZL,Catlins Conservation Park,Catlins Conservation Park,Designated,Federal or national ministry or agency,Stewardship Area,Stewardship Area,National,III,8.412168,"MULTIPOLYGON (((169.30645 -46.57949, 169.30754..."


#### Save file as shp and as csv

In [35]:
gwd = gwd.drop(columns=['index_left'])
# gwd.to_file(filename='/Users/sofia/Documents/HE_Data/wdpa_with_gadm1', driver='ESRI Shapefile')
gwd = gwd.drop(columns=['geometry'])
gwd.to_csv('/Users/sofia/Documents/HE_Data/wdpa_with_gadm1.csv')

In [36]:
gwd.head()

,GID_0,NAME_0,NAME_1,MOL_IDg,MOL_ID,ISO3,NAME,ORIG_NA,STATUS,GOV_TYP,DESIG,DESIG_E,DESIG_T,IUCN_CA,AREA_KM
0,NZL,New Zealand,Otago,2215.0,1,NZL,Boulder Beach,Boulder Beach / WWF Block,Designated,Federal or national ministry or agency,Stewardship Area,Stewardship Area,National,III,1.136031
1,NZL,New Zealand,Marlborough,2211.0,2,NZL,Ferndale,Ferndale,Designated,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.748492
2,NZL,New Zealand,Marlborough,2211.0,3,NZL,Broughton Bay,Broughton Bay,Designated,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.031907
3,NZL,New Zealand,Marlborough,2211.0,4,NZL,Kaipupu Point,Kaipupu Point,Designated,Federal or national ministry or agency,Scenic Reserve,Scenic Reserve,National,III,0.270855
4,NZL,New Zealand,Otago,2215.0,5,NZL,Catlins Conservation Park,Catlins Conservation Park,Designated,Federal or national ministry or agency,Stewardship Area,Stewardship Area,National,III,8.412168


#### Publish layers with API

In [12]:
# Function to publish files as feature services or hosted tables

def publishCSVasFS(csv_file,csvName, gis, aol_folder_name, sharing, table): # sharing = 'None', 'groups', 'everyone' # table = 'yes', 'no'
    try:
        
        if gis.content.is_service_name_available(csvName, "featureService"):
            print(f"Service name {csvName} is available")
            #csv_file = f'./{csvName}.csv'
            csv_item = gis.content.add({}, csv_file)
            #csv_lyr = csv_item.publish(None,  {'CountryCode' : 'countryname'})
            if table == 'yes':
                csv_lyr = csv_item.publish(publish_parameters={'locationType':'none', 'type':'csv'})
            else:
                csv_lyr = csv_item.publish()
                flayer_collection = FeatureLayerCollection.fromitem(csv_lyr)
                searched_flayer = flayer_collection.layers[0] 
                nber_features = searched_flayer.query(return_count_only=True)
                print(f"The service {csvName} has been published. The service has {nber_features} entries")
            print(f"Moving service {csvName} to {aol_folder_name} in ArcGIS Online...")
            # move both the csv_item and csv_lyr items into this new folder
            csv_item.move(aol_folder_name)
            csv_lyr.move(aol_folder_name)
            print(f"Service {csvName} has been moved to {aol_folder_name} in ArcGIS Online")
#             sharing, default is owner
#             csv_lyr.shared_with
            if sharing == 'None':
                csv_lyr.share(everyone=False, org=False, groups=None, allow_members_to_edit=False)
            if sharing == 'groups':
            # the group id is needed to share with agroup
                csv_lyr.share(everyone=False, org=False, groups=[group_id], allow_members_to_edit=False)
            if sharing == 'everyone':
                csv_lyr.share(everyone=True, org=False, groups=None, allow_members_to_edit=False)
            sharing_prop = csv_lyr.shared_with
            if sharing_prop['everyone']==True:
                print(f"shared with everyone")   
            else:
                print(f"not a public layer, for this layer to be used it has to be public or the urls have to be whitelisted")            
            #not allowing deleting
            csv_lyr.protect()
            print(f"{csvName}'s protection against deletion : {csv_lyr.protected}") 
            # it is possible to check the status with csv_item.content_status
            return csv_lyr.id
        else:
            print("The service name is not available, try overwritting, appending the data or a different service name")
    except:
        print("The csv hasn't been published")

In [16]:
# Run function with the file we want to add
csv_file = '/Users/sofia/Documents/HE_Data/wdpa_with_gadm1.zip'
csvName = 'wdpa_with_gadm1'
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)
aol_folder_name = '#6 precalculated data'
group_id='46783c33c294485abe83f5e899c2f21f'
publishCSVasFS(csv_file,csvName,gis,aol_folder_name,table='no',sharing='groups')

Service name wdpa_with_gadm1 is available
The service wdpa_with_gadm1 has been published. The service has 236980 entries
Moving service wdpa_with_gadm1 to #6 precalculated data in ArcGIS Online...
Service wdpa_with_gadm1 has been moved to #6 precalculated data in ArcGIS Online
not a public layer, for this layer to be used it has to be public or the urls have to be whitelisted
wdpa_with_gadm1's protection against deletion : True


'877488f05f104f3ab710e62dda44ae82'